We want to plot the power and gaz network that we consider

In [2]:
import pandas as pd
import numpy as np
import folium
import os
from folium.plugins import MarkerCluster

In [13]:
node_nb = 6

DATA_path = os.path.dirname(os.getcwd())+"/DATA"
Pow_dir = DATA_path+'/Power_System_Data'
Gas_dir = DATA_path+'/NG_System_Data'

NG_nodes = pd.read_csv(Gas_dir+'/NG_Nodes.csv')
NG_pipelines = pd.read_csv(Gas_dir+'/NG2NG_Pipelines.csv')
NG_pipelines = NG_pipelines.loc[NG_pipelines['is_existing'] == 1]

NG_2_power = pd.read_csv(Pow_dir+f"/{node_nb}-Nodes/NG_AdjE_Nodes.csv")
#NG_2_power.columns = ['NG_node', 'Power_node']

Power_nodes = pd.read_csv(Pow_dir+f"/{node_nb}-Nodes"+'/Power_Nodes.csv')
Power_lines = pd.read_csv(Pow_dir+f"/{node_nb}-Nodes"+'/Transmission_Lines.csv')
Power_lines = Power_lines.loc[Power_lines['is_existing'] == 1]

In [14]:
world_map = folium.Map(location=(43, -70), zoom_start=7, tiles="cartodbpositron")

# NG network
for _, row in NG_nodes.iterrows():
    lat, long = row['Lat'], -row['Lon']
    folium.CircleMarker(location=(lat, long), radius=5, color='blue').add_to(world_map)

for _, row in NG_pipelines.iterrows():
    fr_node = NG_nodes.loc[NG_nodes['node_num'] == row['from_node']]
    to_node = NG_nodes.loc[NG_nodes['node_num'] == row['to_node']]
    fr_node_coords = (fr_node['Lat'].values[0], -fr_node['Lon'].values[0])
    to_node_coords = (to_node['Lat'].values[0], -to_node['Lon'].values[0])
    folium.PolyLine(locations=[fr_node_coords, to_node_coords], color='blue').add_to(world_map)

# Power network
for i, row in Power_nodes.iterrows():
    lat, long = row['Lat'], -row['Lon']
    folium.CircleMarker(location=(lat, long), radius=5,  color='red', popup=i).add_to(world_map)

for _, row in Power_lines.iterrows():
    fr_node = Power_nodes.loc[Power_nodes['node_num'] == row['from_node']]
    to_node = Power_nodes.loc[Power_nodes['node_num'] == row['to_node']]
    fr_node_coords = (fr_node['Lat'].values[0], -fr_node['Lon'].values[0])
    to_node_coords = (to_node['Lat'].values[0], -to_node['Lon'].values[0])
    folium.PolyLine(locations=[fr_node_coords, to_node_coords],color='red').add_to(world_map)

"""
# NG to Power
for _, row in NG_2_power.iterrows():
    fr_node = NG_nodes.loc[NG_nodes['node_num'] == row['NG_node']]
    if not row['Power_node']>0:
        continue
    to_node = Power_nodes.loc[Power_nodes['node_num'] == row['Power_node']]
    fr_node_coords = (fr_node['Lat'].values[0], -fr_node['Lon'].values[0])
    to_node_coords = (to_node['Lat'].values[0], -to_node['Lon'].values[0])
    folium.PolyLine(locations=[fr_node_coords, to_node_coords],  color='green').add_to(world_map)
    """


legend_html = '''
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: 150px; height: 120px; 
            border:2px solid grey; z-index:9999; font-size:14px;
            background-color:white; opacity: 0.8;">
    <b>Legend</b><br>
    <span style="display:inline-block; width:12px; height:12px; border: 3px solid blue; border-radius:50%; margin-right:5px;"></span> NG Node<br>
    <span style="display:inline-block; width:12px; height:12px; border: 3px solid red; border-radius:50%; margin-right:5px;"></span> Power Node<br>
    <span style="display:inline-block; width:20px; height:3px; background-color:blue; margin-right:5px;"></span> NG Pipeline<br>
    <span style="display:inline-block; width:20px; height:3px; background-color:red; margin-right:5px;"></span> Power Line<br>
    <span style="display:inline-block; width:20px; height:2px; background-color:green; margin-right:5px;"></span> NG to Power<br>
</div>
'''

world_map.get_root().html.add_child(folium.Element(legend_html))
world_map

### Assignement of each power node to the closest NG node

In [79]:
from geopy.distance import geodesic

# Function to calculate the distance between two points
def calculate_distance(coord1, coord2):
    return geodesic(coord1, coord2).kilometers

# Create a new dataframe to store the closest NG node for each power node
#closest_ng_nodes = pd.DataFrame(columns=['Power_node', 'Closest_NG_node'])

NG = [(row['Lat'], -row['Lon']) for _, row in NG_nodes.iterrows()]

closest_Power_node = np.zeros(len(NG))

# Iterate through each power node
for _, power_node in Power_nodes.iterrows():
    power_node_coords = (power_node['Lat'], -power_node['Lon'])
    min_distance = float('inf')
    closest_ng_node = None

    dist = [calculate_distance(power_node_coords, ng) for ng in NG]
    closest_ng = np.argmin(dist)

    closest_Power_node[closest_ng] = _

df = pd.DataFrame(np.transpose(np.array([np.arange(len(NG)), closest_Power_node])))
df.to_csv('NG_AdjE_Nodes.csv', index=False)